In [1]:
import plotly.plotly
import plotly.graph_objs as go
from plotly.offline import *

In [20]:
class Node(object):
    def __init__(self, n=1, x=0, y=0, z=0):
        self.position = [int(x),int(y),int(z)]
        self.birthday = n
        self.parent = None
        self.children = []
    
    def setBirthday(self,n):
        self.birthday = n
        
    def getBirthday(self):
        return self.birthday
    
    def setParent(self, parent):
        self.parent = parent
    
    def getParent(self):
        return self.parent
    
    def addChild(self, child):
        child.setParent(self)
        self.children.append(child)
    
    def getChildren(self):
        return self.children
    
    def setPosition(self, x,y,z):
        self.position = [x,y,z]
        
    def getPosition(self):
        return self.position
    
    def equals(self, node):
        return (self.position == node.position)
    
    def preBirth(self):
        n = self.birthday
        x = self.position[0]
        y = self.position[1]
        z = self.position[2]
        
        A = Node(n+1, x+1, y+1, z+(-1)**x)
        B = Node(n+1, x-1, y-1, z+(-1)**x)
        C = Node(n+1, x+1, y-1, z+(-1)**(x+1))
        D = Node(n+1, x-1, y+1, z+(-1)**(x+1))
        
        A.setParent(self)
        B.setParent(self)
        C.setParent(self)
        D.setParent(self)
        return [A,B,C,D]
    
    def get3Dpoints(self):
        x = self.position[0]
        y = self.position[1]
        z = self.position[2]
        if x%2 == 1:
            points = [  [x+0,   y+0,   z+0],
                        [x-0.5, y-0.5, z-0.5],
                        [x-0.5, y+0.5, z+0.5],
                        [x+0.5, y-0.5, z+0.5],
                        [x+0.5, y+0.5, z-0.5]]
        else:
            points = [  [x+0,   y+0,   z+0],
                        [x+0.5, y-0.5, z-0.5],
                        [x+0.5, y+0.5, z+0.5],
                        [x-0.5, y-0.5, z+0.5],
                        [x-0.5, y+0.5, z-0.5]]
        return points
    
    def getFull3Dpoints(self):
        points = []
        points += self.get3Dpoints()
        for child in self.children:
            points += child.getFull3Dpoints()
        return points
    
    def getFull3DpointsGroup(self,n = 0):
        groups = []
        points = []
        points += self.get3Dpoints()
        for i in range(5):
            groups += [n]
        for child in self.children:
            data = child.getFull3DpointsGroup(n+1)
            points += data[0]
            groups += data[1]
        return [points,groups]
            

In [21]:
class TreeClass(object):
    def __init__(self,root):
        self.data = [[root]]
        self.population = 1
        self.root = root
    
    def append(self,a):
        self.population += len(a)
        self.data.append(a)
        
    def getData(self):
        
        return self.data
        
    def getDepth(self):
        return len(self.data)
    
    def getLastBranch(self):
        return self.data[self.getDepth()-1]
    
    def getPopulation(self):
        return self.population
        
    def get3Dpoints(self):
        return self.root.getFull3Dpoints()
        
    def get3DpointsGroup(self):
        return self.root.getFull3DpointsGroup()
    
    def printData(self):
        print("Total population: ",self.getPopulation())
        for level in self.data:
            for node in level:
                pos = node.getPosition()
                print("n: ",node.getBirthday()," pos:(",pos[0],", ",pos[1],", ",pos[2],")")

In [22]:
def BloomBF(Tree):
    preNodes = []
    newNodes = []
    for node in Tree.getLastBranch():
        preNodes.extend(node.preBirth())
    
    for i in range(len(preNodes)):
        repeated = False
        for branchLevel in Tree.getData():
            for node in branchLevel:
                if preNodes[i].equals(node):
                    repeated = True
        for j in range(len(preNodes)):
            if i != j and preNodes[i].equals(preNodes[j]):
                repeated = True
        if repeated == False:
            preNodes[i].getParent().addChild(preNodes[i])
            newNodes.append(preNodes[i])
    
    Tree.append(newNodes)
    return Tree

In [84]:
root = Node()
Tree = TreeClass(root)
Tree.printData()

Total population:  1
n:  1  pos:( 0 ,  0 ,  0 )


In [85]:
n = 16
bbs = [Tree.getPopulation()] # Baby Breath Sequence
for i in range(n-1):
    Tree = BloomBF(Tree)
    bbs.append(Tree.getPopulation())

In [86]:
print("Baby Breath Sequence: ",bbs)

Baby Breath Sequence:  [1, 5, 17, 29, 41, 77, 161, 197, 209, 245, 353, 533, 665, 869, 1313, 1469]


In [87]:
Tree.getData()[2][2].getPosition()

[0, 2, 2]

In [88]:
'''
Points = Tree.get3Dpoints()
EdgesList = []
group = []
for i in range(Tree.getPopulation()):
    Edges = []
    for j in range(4):
        Edges.append([i*5+0,i*5+j+1])
    EdgesList.append(Edges)
'''

'\nPoints = Tree.get3Dpoints()\nEdgesList = []\ngroup = []\nfor i in range(Tree.getPopulation()):\n    Edges = []\n    for j in range(4):\n        Edges.append([i*5+0,i*5+j+1])\n    EdgesList.append(Edges)\n'

In [89]:
data = Tree.get3DpointsGroup()
Points = data[0]
group = data[1]
EdgesList = []
for i in range(Tree.getPopulation()):
    Edges = []
    for j in range(4):
        Edges.append([i*5+0,i*5+j+1])
    EdgesList.append(Edges)

In [90]:
Xn=[Points[k][0] for k in range(len(Points))]# x-coordinates of nodes
Yn=[Points[k][1] for k in range(len(Points))]# y-coordinates of nodes
Zn=[Points[k][2] for k in range(len(Points))]# z-coordinates of nodes

In [91]:
PlotData = []

In [92]:
'''
group = []
g = 0
for i in bbs:
    for j in range(5*i-len(group)):
        group += [g]
    g+=1
'''

'\ngroup = []\ng = 0\nfor i in bbs:\n    for j in range(5*i-len(group)):\n        group += [g]\n    g+=1\n'

In [93]:
#Points

In [94]:
#EdgesList

In [95]:
trace=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=dict(symbol='circle',
                             size=6,
                             color=group,
                             colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               text=[],
               hoverinfo='text'
               )
PlotData.append(trace)

In [96]:
for Edges in EdgesList:
    Xe=[]
    Ye=[]
    Ze=[]
    for e in Edges:
        Xe+=[Points[e[0]][0],Points[e[1]][0]]# x-coordinates of edge ends
        Ye+=[Points[e[0]][1],Points[e[1]][1]]  
        Ze+=[Points[e[0]][2],Points[e[1]][2]]  

    trace2=go.Scatter3d(x=Xe,
                   y=Ye,
                   z=Ze,
                   mode='lines',
                   line=dict(color='rgb(125,125,125)', width=3),
                   hoverinfo='none'
                   )
    PlotData.append(trace2)

In [97]:
axis=dict(showbackground=True,
          showline=True,
          zeroline=True,
          showgrid=True,
          showticklabels=True,
          title=''
          )

layout = go.Layout(
         title="Babybreath graph",
         width=1000,
         height=1000,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',    )

In [98]:
fig=go.Figure(data=PlotData, layout=layout)
plotly.offline.plot(fig, filename = 'graph'+str(n)+'.html')

'file://C:\\Users\\ediaz\\Documents\\PrograJAVA\\BabyBreath\\BabyBreath\\graph16.html'